# Model v3
Anthony Lee 2025-01-03

Improvement TODO:
- Scale input to [-1, 1] range from [0, 255]
- Add a tanh at the end of the upsampling stack of the generator
- Interested in using a pre-trained UNET as the generator


In [ ]:
from gan.gan import CycleGAN, train_one_epoch, checkpoint_save
from gan.data import ImageDataset, ImageDataLoader
import torch
from pprint import pprint
from gan.plotting_utility import plot_before_after

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print( torch.cuda.get_device_properties(device) )


monet_dataset = ImageDataset(data_dir="../data/gan-getting-started/monet_jpg")
photo_dataset = ImageDataset(data_dir="../data/gan-getting-started/photo_jpg")

monet_dataloader = ImageDataLoader(monet_dataset)
photo_dataloader = ImageDataLoader(photo_dataset)


In [ ]:
model = CycleGAN()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4, betas=(0.5, 0.999))

for idx_epoch in range(20):

    loss_tracker = train_one_epoch(monet_dataloader=monet_dataloader, 
                    photo_dataloader=photo_dataloader,
                    optimizer=optimizer,
                    model=model, 
                    device=device)
    
    ## Plot an example image
    idx = 20

    # Photo to monet
    input = photo_dataset[idx]
    output = model.generate_monet(input)
    fig = plot_before_after(input, output, suptitle=f"Epoch {idx_epoch}: Photo -> Monet")
    fig.show()

    # Monet to photo
    input = monet_dataset[idx]
    output = model.generate_photo(input)
    fig = plot_before_after(input, output, suptitle=f"Epoch {idx_epoch}: Monet -> Photo")
    fig.show()

    ## Checkpoint
    _ = checkpoint_save(epoch=idx_epoch, 
                save_path=".", 
                model=model,
                optimizer=optimizer,
                loss_tracker=loss_tracker)
    